# Dataset Preparation

In [5]:
!pip install python-terrier -q

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from IPython.display import display
from xml.dom.minidom import parse, parseString
from pyterrier.measures import *

import random
import pyterrier as pt
import pandas as pd
import json
import os
import re
from tqdm import tqdm
tqdm.pandas()

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



# Doc Expand

In [8]:
df_beam_5 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_5_cutted_version.xlsx")
df_beam_10 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_10_cutted_version.xlsx")
df_random_5 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_5_cutted_version.xlsx")
df_random_10 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_5_cutted_version.xlsx")
df_beam_7 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_7_cutted_version.xlsx")
df_random_7 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_7_cutted_version.xlsx")


In [9]:
df_beam_5.dropna(subset=['docno'], inplace=True)
df_beam_5.fillna('', inplace=True)
df_beam_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [10]:
df_beam_10.dropna(subset=['docno'], inplace=True)
df_beam_10.fillna('', inplace=True)
df_beam_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [11]:
df_random_5.dropna(subset=['docno'], inplace=True)
df_random_5.fillna('', inplace=True)
df_random_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [12]:
df_random_10.dropna(subset=['docno'], inplace=True)
df_random_10.fillna('', inplace=True)
df_random_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [13]:
df_beam_7.dropna(subset=['docno'], inplace=True)
df_beam_7.fillna('', inplace=True)
df_beam_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [14]:
df_random_7.dropna(subset=['docno'], inplace=True)
df_random_7.fillna('', inplace=True)
df_random_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


# Indexing

In [15]:
!rm -rf ./coliee_beam_5_index
pd_indexerb5 = pt.DFIndexer("./coliee_beam_5_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam5 = pd_indexerb5.index(df_beam_5["text"], df_beam_5)

!rm -rf ./coliee_beam_10_index
pd_indexerb10 = pt.DFIndexer("./coliee_beam_10_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam10 = pd_indexerb10.index(df_beam_10["text"], df_beam_10)

!rm -rf ./coliee_beam_7_index
pd_indexerb7 = pt.DFIndexer("./coliee_beam_7_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam7 = pd_indexerb7.index(df_beam_7["text"], df_beam_7)

!rm -rf ./coliee_random_5_index
pd_indexerr5 = pt.DFIndexer("./coliee_random_5_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand5 = pd_indexerr5.index(df_random_5["text"], df_random_5)

!rm -rf ./coliee_random_10_index
pd_indexerr10 = pt.DFIndexer("./coliee_random_10_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand10 = pd_indexerr10.index(df_random_10["text"], df_random_10)

!rm -rf ./coliee_random_7_index
pd_indexerr7 = pt.DFIndexer("./coliee_random_7_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand7 = pd_indexerr7.index(df_random_7["text"], df_random_7)



In [16]:
df_query = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/df_query.xlsx")
articles = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/df_articles.xlsx")

In [17]:
articles.dropna(subset=['docno'], inplace=True)
articles.fillna('', inplace=True)
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 776 entries, 0 to 780
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        776 non-null    object
 1   docno       776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 48.5+ KB


In [18]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1402 entries, 0 to 1401
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   qid       1402 non-null   object
 1   query     1402 non-null   object
 2   entail    1402 non-null   object
 3   label     1402 non-null   int64 
 4   art       1402 non-null   object
 5   art_code  1402 non-null   object
dtypes: int64(1), object(5)
memory usage: 65.8+ KB


In [19]:
topics = df_query[["qid", "query"]].drop_duplicates()
topics['category'] = topics.apply(lambda row: row.qid.split('-')[0], axis = 1)
topics_R04 = topics[topics["category"] == 'R04']

# creating qrels ---> all articles in the training data are relevant; all others
# are assumed to be non-relevant (including negative samples)
qrels_temp = df_query[["qid", "art_code", "label"]].copy()
qrels = qrels_temp.rename(columns={"art_code": "docno"})

In [20]:
!rm -rf ./coliee_index

# we need to use blocks = True
# All indexer classes expose a blocks boolean constructor argument to allow position information to be
# recoreded in the index. Defaults to False, i.e. positions are not recorded.
# Indexing will record "position" aka blocks information, however this makes the indices much larger and also slower.
pd_indexer = pt.DFIndexer("./coliee_index", type=pt.index.IndexingType(1), blocks = True)

# note that we also keep the articles as "text" in the metadata part
# (Add the entire dataframe as metadata)
# this will be useful when we re-rank model using LLM, such as T5, BERT, etc.
index_ref = pd_indexer.index(articles["text"], \
                             articles["text"], \
                             articles["docno"], \
                             articles["part"], \
                             articles["chap"], \
                             articles["sect"], \
                             articles["subsect"], \
                             articles["subsubsect"])

In [21]:
bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
tfidf = pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30

b5bm25 = pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25")% 30
b5tf = pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
b10bm25 = pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25")% 30
b10tf = pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
b7bm25 = pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25")% 30
b7tf = pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

r5bm25 = pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25")% 30
r5tf = pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
r10bm25 = pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25")% 30
r10tf = pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
r7bm25 = pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25")% 30
r7tf = pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30


In [22]:
eksperimen =pt.Experiment(
    [bm25,
     tfidf,
     b5bm25,
     b10bm25,
     b7bm25,
     b5tf,
     b10tf,
     b7tf,
     r5bm25,
     r10bm25,
     r7bm25,
     r5tf,
     r10tf,
     r7tf,
     ],
    topics_R04,
    qrels,
    eval_metrics= [P@5, R@10, R@15, R@20, "recip_rank"],
    verbose=True,
    names=["BM25","TFIDF","Beam-5-BM25", "Beam-10-BM25", "Beam-7-BM25","Beam-5-TFIDF", "Beam-10-TFIDF","Beam-7-TFIDF","Random-5-BM25", "Random-10-BM25","Random-7-BM25", "Random-5-TFIDF", "Random-10-TFIDF", "Random-7-TFIDF"]
)

pt.Experiment:   0%|          | 0/14 [00:00<?, ?system/s]

In [23]:
eksperimen

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TFIDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,Beam-5-BM25,0.186139,0.861386,0.876238,0.900990,0.742088
3,Beam-10-BM25,0.190099,0.851485,0.881188,0.891089,0.739803
4,Beam-7-BM25,0.188119,0.846535,0.871287,0.876238,0.752977
5,Beam-5-TFIDF,0.190099,0.856436,0.876238,0.891089,0.741401
6,Beam-10-TFIDF,0.190099,0.846535,0.881188,0.881188,0.741388
7,Beam-7-TFIDF,0.188119,0.846535,0.876238,0.876238,0.746107
8,Random-5-BM25,0.184158,0.851485,0.876238,0.896040,0.741055
9,Random-10-BM25,0.184158,0.851485,0.876238,0.896040,0.741055
